In [ ]:
import os
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

print("✓ Environment ready")


In [4]:
DATA_DIR = "D:/Samvad_Setu_final/datasets/ASL Dataset/asl_alphabet_train/asl_alphabet_train"

classes = sorted(os.listdir(DATA_DIR))
print(classes)
print("Total classes:", len(classes))


['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
Total classes: 29


In [5]:
import cv2
import mediapipe as mp
import numpy as np

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Initialize MediaPipe ONCE (not inside loop)
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    model_complexity=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

def extract_landmarks(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = hands.process(image_rgb)

    if not result.multi_hand_landmarks:
        return None

    lm = []
    for p in result.multi_hand_landmarks[0].landmark:
        lm.extend([p.x, p.y, p.z])

    return np.array(lm, dtype=np.float32)


---

In [8]:
X = []
Y = []

for label in classes:
    folder_path = os.path.join(DATA_DIR, label)

    print(f"Processing: {label} ...")

    for file in os.listdir(folder_path):
        if not file.lower().endswith((".png", ".jpg", ".jpeg")):
            continue
        
        img_path = os.path.join(folder_path, file)
        lm = extract_landmarks(img_path)

        if lm is not None:
            X.append(lm)
            Y.append(label)

X = np.array(X)
Y = np.array(Y)

print("✓ Finished dataset extract!")
print("Dataset shape:", X.shape, Y.shape)


Processing: A ...
Processing: B ...
Processing: C ...
Processing: D ...
Processing: E ...
Processing: F ...
Processing: G ...
Processing: H ...
Processing: I ...
Processing: J ...
Processing: K ...
Processing: L ...
Processing: M ...
Processing: N ...
Processing: O ...
Processing: P ...
Processing: Q ...
Processing: R ...
Processing: S ...
Processing: T ...
Processing: U ...
Processing: V ...
Processing: W ...
Processing: X ...
Processing: Y ...
Processing: Z ...
Processing: del ...
Processing: nothing ...
Processing: space ...
✓ Finished dataset extract!
Dataset shape: (63676, 63) (63676,)


In [1]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Encode labels
le = LabelEncoder()
Y_encoded = le.fit_transform(Y)
Y_onehot = to_categorical(Y_encoded)

# Normalize landmarks → VERY IMPORTANT for LSTM
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# reshape -> (samples, 63, 1)
X_scaled = X_scaled.reshape(-1, 63, 1)

# split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, Y_onehot, test_size=0.2, random_state=42, shuffle=True
)

print("Train:", X_train.shape, " Test:", X_test.shape)


NameError: name 'Y' is not defined

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("asl_lstm_model.h5")
model = load_model("asl_lstm_model.h5")

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, LayerNormalization

model = Sequential([
    Bidirectional(LSTM(128, return_sequences=True), input_shape=(63,1)),
    LayerNormalization(),
    Dropout(0.3),

    Bidirectional(LSTM(64)),
    LayerNormalization(),
    Dropout(0.3),

    Dense(128, activation="relu"),
    Dropout(0.25),

    Dense(len(classes), activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


In [8]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical



images = []
labels = []

classes = sorted(os.listdir(DATA_DIR))   # A-Z classes
print("Classes found:", classes)

for label in classes:
    folder = os.path.join(DATA_DIR, label)

    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (63, 63))

        images.append(img)
        labels.append(classes.index(label))

images = np.array(images).reshape(-1, 63, 63, 1)
labels = np.array(labels)

print("Dataset loaded!")
print("X shape:", images.shape)
print("y shape:", labels.shape)


Classes found: ['asl_alphabet_train']
Skipping corrupted file: ../datasets/ASL Dataset/asl_alphabet_train\asl_alphabet_train\A
Skipping corrupted file: ../datasets/ASL Dataset/asl_alphabet_train\asl_alphabet_train\B
Skipping corrupted file: ../datasets/ASL Dataset/asl_alphabet_train\asl_alphabet_train\C
Skipping corrupted file: ../datasets/ASL Dataset/asl_alphabet_train\asl_alphabet_train\D
Skipping corrupted file: ../datasets/ASL Dataset/asl_alphabet_train\asl_alphabet_train\del
Skipping corrupted file: ../datasets/ASL Dataset/asl_alphabet_train\asl_alphabet_train\E
Skipping corrupted file: ../datasets/ASL Dataset/asl_alphabet_train\asl_alphabet_train\F
Skipping corrupted file: ../datasets/ASL Dataset/asl_alphabet_train\asl_alphabet_train\G
Skipping corrupted file: ../datasets/ASL Dataset/asl_alphabet_train\asl_alphabet_train\H
Skipping corrupted file: ../datasets/ASL Dataset/asl_alphabet_train\asl_alphabet_train\I
Skipping corrupted file: ../datasets/ASL Dataset/asl_alphabet_train\as

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=7,
    restore_best_weights=True
)

lr_reduce = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=3,
    min_lr=1e-6
)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=64,
    callbacks=[early_stop, lr_reduce]
)


In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", acc)


In [10]:
import pickle

model.save("asl_lstm_model.h5")
pickle.dump(le, open("label_encoder.pkl", "wb"))
pickle.dump(scaler, open("scaler.pkl", "wb"))

print("✓ Model, Label Encoder & Scaler saved")


✓ Model and LabelEncoder saved


---

In [ ]:
from tensorflow.keras.models import load_model
import pickle
import cv2
import numpy as np
import mediapipe as mp

# load assets
model = load_model("asl_lstm_model.h5")
le = pickle.load(open("label_encoder.pkl", "rb"))
scaler = pickle.load(open("scaler.pkl", "rb"))

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret: break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(frame_rgb)

    if result.multi_hand_landmarks:
        lm = []
        for p in result.multi_hand_landmarks[0].landmark:
            lm.extend([p.x, p.y, p.z])

        lm = np.array(lm).reshape(1, -1)

        # scale
        lm_scaled = scaler.transform(lm).reshape(1, 63, 1)

        # prediction
        pred = model.predict(lm_scaled, verbose=0)
        label = le.inverse_transform([np.argmax(pred)])[0]

        cv2.putText(frame, label, (30, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0), 2)

    cv2.imshow("ASL Live", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


---